## Start Up and DF Building

In [1]:
import numpy as np
import pandas as pd
import pickle as pkl

from keras.optimizers import Adam
from keras.layers import BatchNormalization, Conv2D, Conv3D, ConvLSTM2D, Dense, Dropout, Flatten, LSTM, MaxPooling2D, MaxPooling3D, TimeDistributed
from keras.models import Sequential
from keras.utils import to_categorical

from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

DATA_DIR = (Path().parent / "../data").resolve()
DATA_DIR

WindowsPath('C:/Users/Michael/Downloads/Final-Year-Project/data')

In [2]:
raw_data: pd.DataFrame = pd.read_pickle(DATA_DIR / "dataset.pkl")
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   file_name  50 non-null     object  
 1   sign       50 non-null     category
 2   left       50 non-null     object  
 3   right      50 non-null     object  
 4   hand       50 non-null     category
dtypes: category(2), object(3)
memory usage: 1.5+ KB


In [3]:
raw_data[["sign", "hand"]].value_counts()

sign   hand 
when   both     10
hello  left      5
       right     5
no     left      5
       right     5
okay   left      5
       right     5
yes    left      5
       right     5
dtype: int64

## Preparing Data for Splitting

In [4]:
def get_data(left: np.ndarray, right: np.ndarray, max_frames: int) -> np.ndarray:
    if (len(left) < max_frames):
        increment = [np.zeros(shape=441) for _ in range(max_frames - len(left))]
        temp_1 = np.concatenate((left, increment), axis=0)
        temp_2 = np.concatenate((right, increment), axis=0)
        result = np.stack((temp_1, temp_2), axis=0)

    else:
        result = np.stack((left, right), axis=0)

    # return result.reshape((2,max_frames,441,1))
    return result

In [5]:
max_frames = max(len(item) for item in raw_data["left"])
print(f"Max Frame Count: {max_frames}")

X = np.array([
    get_data(raw_data.loc[x]["left"], raw_data.loc[x]["right"], max_frames)
    for x in range(len(raw_data))
])

X.shape

Max Frame Count: 78


(50, 2, 78, 441)

In [6]:
lbl_enc = LabelEncoder()
labels = lbl_enc.fit_transform(raw_data["sign"])

# with open(DATA_DIR / "label_encoder.pkl", "wb") as file:
#     pkl.dump(obj=lbl_enc, file=file)

Y = to_categorical(labels)
Y.shape

(50, 5)

## Spliting Data

In [7]:
train_data, test_data, train_labels, test_labels = train_test_split(X, Y, test_size=0.2, random_state=42, stratify=Y)

In [8]:
train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=0.25, random_state=42, stratify=train_labels)

In [9]:
train_data.shape

(30, 2, 78, 441)

In [10]:
train_labels.shape

(30, 5)

In [11]:
val_data.shape

(10, 2, 78, 441)

In [12]:
val_labels.shape

(10, 5)

In [13]:
test_data.shape

(10, 2, 78, 441)

In [14]:
test_labels.shape

(10, 5)

## Model Building

### Model 01

In [15]:
# model = Sequential()
# model.add(Conv3D(32, kernel_size=(1, 3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=train_data.shape[1:]))
# model.add(MaxPooling3D(pool_size=(2, 2, 2)))
# model.add(BatchNormalization(center=True, scale=True))
# model.add(Dropout(0.5))

# model.add(Conv3D(64, kernel_size=(1, 3, 3), activation='relu', kernel_initializer='he_uniform'))
# model.add(MaxPooling3D(pool_size=(1, 2, 2)))
# model.add(BatchNormalization(center=True, scale=True))
# model.add(Dropout(0.5))
# model.add(Flatten())

# model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
# model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
# model.add(Dense(train_labels.shape[1], activation='softmax'))

### Model 02

In [16]:
# model = Sequential()
# model.add(Conv3D(32, kernel_size=(1, 3, 3), activation='relu', input_shape=train_data.shape[1:]))
# model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# model.add(Conv3D(64, kernel_size=(1, 3, 3), activation='relu'))
# model.add(MaxPooling3D(pool_size=(1, 2, 2)))
# model.add(Flatten())

# model.add(Dense(128, activation='relu'))
# model.add(Dense(train_labels.shape[1], activation='softmax'))

### Model 03

In [17]:
# model = Sequential()
# model.add(Conv3D(32, kernel_size=(1, 3, 3), activation='relu', input_shape=train_data.shape[1:]))
# model.add(MaxPooling3D(pool_size=(2, 2, 2)))
# model.add(Flatten())

# model.add(Dense(64, activation='relu'))
# model.add(Dense(train_labels.shape[1], activation='sigmoid'))

### Model 04

In [18]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=train_data.shape[1:], padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dense(train_labels.shape[1], activation='softmax'))

### Model 05

In [19]:
# model = Sequential()
# model.add(TimeDistributed(Conv2D(32, kernel_size=(3, 3), activation='relu'), input_shape=train_data.shape[1:]))
# model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

# model.add(TimeDistributed(Conv2D(64, kernel_size=(3, 3), activation='relu')))
# model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

# model.add(TimeDistributed(Conv2D(128, kernel_size=(3, 3), activation='relu')))
# model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

# model.add(TimeDistributed(Conv2D(256, kernel_size=(3, 3), activation='relu')))
# model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
# model.add(TimeDistributed(Flatten()))

# model.add(LSTM(128, activation='tanh', dropout=0.5))
# model.add(Dense(train_labels.shape[1], activation='softmax'))

### Results

In [20]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=0.001),
    metrics=['accuracy']
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 2, 78, 32)         127040    
                                                                 
 max_pooling2d (MaxPooling2D  (None, 1, 39, 32)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 1, 39, 64)         18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 1, 20, 64)        0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 1, 20, 128)        73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 1, 10, 128)       0

In [21]:
history = model.fit(
    x=train_data,
    y=train_labels,
    batch_size=32,
    epochs=10,
    validation_data=(val_data, val_labels),
    verbose=1
)

Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: 1.6139 - accuracy: 0.2000 - val_loss: 1.6016 - val_accuracy: 0.2000
Epoch 2/10
1/1 [==============================] - 0s 154ms/step - loss: 1.5438 - accuracy: 0.3667 - val_loss: 1.5964 - val_accuracy: 0.4000
Epoch 3/10
1/1 [==============================] - 0s 155ms/step - loss: 1.4005 - accuracy: 0.5333 - val_loss: 1.5969 - val_accuracy: 0.3000
Epoch 4/10
1/1 [==============================] - 0s 144ms/step - loss: 1.1968 - accuracy: 0.6667 - val_loss: 1.6376 - val_accuracy: 0.1000
Epoch 5/10
1/1 [==============================] - 0s 159ms/step - loss: 0.9765 - accuracy: 0.7667 - val_loss: 1.7846 - val_accuracy: 0.1000
Epoch 6/10
1/1 [==============================] - 0s 156ms/step - loss: 0.7884 - accuracy: 0.7667 - val_loss: 2.0290 - val_accuracy: 0.2000
Epoch 7/10
1/1 [==============================] - 0s 274ms/step - loss: 0.6451 - accuracy: 0.8000 - val_loss: 2.3512 - val_accuracy: 0.4000
Epoch 8/10
1/1 [=======

In [22]:
test_loss, test_acc = model.evaluate(test_data, test_labels)

print(f"Test loss: {test_loss}")
print(f"Test accuracy: {test_acc}")

1/1 [==============================] - 0s 70ms/step - loss: 2.5838 - accuracy: 0.7000
Test loss: 2.583761692047119
Test accuracy: 0.699999988079071


In [23]:
# model.save(DATA_DIR / 'model.h5')